# LSTM Model Introspection

Utilities for inspecting the Component X LSTM classifier. Run the cells below to print the module structure and a layer-by-layer summary.

In [2]:
%pip install --quiet torchinfo

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/Othercomputers/Mi portátil (1)/Documents/Proyectos/proyecto-grado/01-repos/prevenport-model
!ls


Mounted at /content/drive
/content/drive/Othercomputers/Mi portátil (1)/Documents/Proyectos/proyecto-grado/01-repos/prevenport-model
artifacts  docs			 mlruns     README.md	      src
data	   machine-learning-ops  notebooks  requirements.txt


In [3]:
#%pip install --quiet -e .

import sys
from pathlib import Path

# ensure project root (the directory that contains "src/") is on sys.path
cwd = Path.cwd()
for p in [cwd] + list(cwd.parents):
    if (p / "src").is_dir():
        sys.path.insert(0, str(p))
        break

import torch
from torchinfo import summary

from src.models.lstm_classifier import LSTMClassifier, LSTMClassifierConfig

INPUT_FEATURES = 105  # update if feature transformer changes
SEQ_LEN = 128

config = LSTMClassifierConfig(
    input_size=INPUT_FEATURES,
    hidden_size=128,
    num_layers=2,
    bidirectional=True,
    dropout=0.1,
    num_classes=5,
    pooling="last",
)
model = LSTMClassifier(config)
print(model)

LSTMClassifier(
  (lstm): LSTM(105, 128, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=64, out_features=5, bias=True)
  )
)


In [4]:
summary(
    model,
    input_size=(1, SEQ_LEN, INPUT_FEATURES),
    col_names=("input_size", "output_size", "num_params"),
    depth=3,
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
LSTMClassifier                           [1, 128, 105]             [1, 5]                    --
├─LSTM: 1-1                              [1, 128, 105]             [1, 128, 256]             635,904
├─Sequential: 1-2                        [1, 256]                  [1, 5]                    --
│    └─Linear: 2-1                       [1, 256]                  [1, 128]                  32,896
│    └─ReLU: 2-2                         [1, 128]                  [1, 128]                  --
│    └─Dropout: 2-3                      [1, 128]                  [1, 128]                  --
│    └─Linear: 2-4                       [1, 128]                  [1, 64]                   8,256
│    └─ReLU: 2-5                         [1, 64]                   [1, 64]                   --
│    └─Dropout: 2-6                      [1, 64]                   [1, 64]                   --
│    └─Linear: 2-7     